Experiment Type 1 : Regression , Dirichelet; alpha=1  

In [ ]:
import sys
sys.path.append("..")
from flwr_datasets.partitioner import DirichletPartitioner
import torch.nn as nn
from torch.optim import SGD
from moxi import create_experiment

/home/sgtec/miniconda3/envs/moxifl/lib/python3.11/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


In [ ]:
config = {
  "network_name": "federated_mnist_experiment",
  "network_type": "decentralised",
  "model_type": "parametric",
  "ml_framework": "pytorch",
  "comms": "async",
  "federated_rounds": 5,
  "network_size": 5,
  "metrics": ["mean_perfomance","convergence"],
  "logger": "mlflow",
  "node_base_config":{
      "model":None,
      "learning_rate": 1e-3,
      "optimizer": SGD,
      "train_data":None,
      "val_data":None,
      "criterion":nn.MSELoss,
      "random_sampling":False,
      "n_epochs": 5,
      "batch_size": 16
  },
  "experiment_config":{
      "task": "regression",
      "partitioner": DirichletPartitioner,
      "alpha": 1,
      "num_worker": 2,
      "max_samples": 1000
  },
  "adjcency_matrix": None,
  "model": None }

In [ ]:

class RegressionModel(nn.Module):
    def __init__(self):
        super(RegressionModel, self).__init__()
        self.act_func = nn.ReLU()
        self.layer1 = nn.Linear(3, 24)  # 3 input features
        self.layer2 = nn.Linear(24, 1)  # 1 output

    def forward(self, x):
        x = self.act_func(self.layer1(x))
        x = self.layer2(x)
        return x

In [ ]:
model = RegressionModel()

In [ ]:
flnw = create_experiment(config, model)


In [ ]:
async def execute_experiment(flnw, num_rounds:int, epochs_per_round:int, experiment_name:str):
    await flnw.train(num_rounds= num_rounds, epochs_per_round=epochs_per_round, experiment_name=experiment_name)
    print("Experiment completed.")

In [ ]:
import nest_asyncio
nest_asyncio.apply()
await execute_experiment(flnw, num_rounds=20, epochs_per_round=5, experiment_name="federated_mnist_experiment")